In [1]:
#Import packages
import pandas as pd
import numpy as np
import re
import warnings
import math
import datetime
warnings.filterwarnings('ignore')

## Serial Cardiac Data Preprocessing

In [2]:
#IMPORT FILES NEEDED
shockOrig = pd.read_csv("../Data Validation/Serial Cardiac Caths/Raw Data/Serial Cardiac.csv", sep=",", index_col='PAT_ID').sort_index()
cath = pd.read_csv("../Data Validation/Serial Cardiac Caths/Raw Data/Cath Data.csv", sep=",", index_col='PAT_ID').sort_index()

shockOrig

,PAT_SEX,PAT_DOB,IndexAdmissionCSN,IndexAdmitDt,IndexDischDt,IndexCathCount,Deceased,H2H,SystolicHF,DiastolicHF,...,PA_SYS,PA_DIA,PA_MEAN,PCW_MEAN,FICK_HR,FICKCO,FICKCI,TDCO,TDCI,PA_SAT
PAT_ID,,,,,,,,,,,,,,,,,,,,,
Z103426,F,1970-10-02,2.000000e+11,2020-01-09,4/1/20,2,1,NaN,1.0,NaN,...,56; 59,33; 33,40; 43,25,113.0,2.39,1.42,2.3,1.37,28
Z103697,F,1960-07-10,6.759402e+07,2016-09-12,9/24/16,2,1,NaN,1.0,NaN,...,67,36,48,36,NaN,2.9,1.52,2.99,1.57,41
Z1050590,F,1968-03-26,2.000000e+11,2020-12-02,12/2/20,2,1,NaN,NaN,NaN,...,101,39,62,10; 14; 8,100.0,3.8,2.22,5.17,3.02,59.1
Z1058186,M,1937-06-15,5.383417e+07,2014-09-25,10/11/14,2,2,NaN,1.0,NaN,...,56; 40,27; 16,37; 27,14; 28; 19,NaN,3.3800; 6.0800,1.8300; 3.2900,3.1800; 4.7200,1.7200; 2.5500,52.0000; 69.0000
Z1059364,F,1957-09-24,5.440898e+07,2014-10-27,11/20/14,2,1,NaN,1.0,NaN,...,50,17,33,18,NaN,4.35,2.16,4.16,2.07,63
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Z975360,M,1956-02-23,2.000000e+11,2018-01-18,3/26/18,5,1,NaN,NaN,1.0,...,67,34,49,37; 36,NaN,2.89,1.31,3.07,1.4,43
Z98220,M,1972-10-03,2.000000e+11,2019-08-28,9/4/19,3,1,NaN,1.0,NaN,...,91; 93,50; 55,64; 70,26,87.0,3.11,1.27,1.97,0.8,37.3
Z984151,M,1981-09-07,2.000000e+11,2019-02-25,3/27/19,3,1,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
# # Find indexes from shock that are not cath
np.setdiff1d(shockOrig.index, cath.index)

array(['Z2802191', 'Z394907'], dtype=object)

In [4]:
#For initial training, just use first row for each patient
shock = pd.DataFrame(columns=shockOrig.columns)
print("Num indexes = ", len(set(shockOrig.index)))

for idx in set(shockOrig.index):
    pt = shockOrig.loc[idx]
    if type(pt) != pd.Series:
#         print(pt)
        row = pt.iloc[0]
        shock = shock.append(row)
    else:
        shock = shock.append(pt)

shockOrig = shock.sort_index()

shockOrig

Num indexes =  267


,PAT_SEX,PAT_DOB,IndexAdmissionCSN,IndexAdmitDt,IndexDischDt,IndexCathCount,Deceased,H2H,SystolicHF,DiastolicHF,...,PA_SYS,PA_DIA,PA_MEAN,PCW_MEAN,FICK_HR,FICKCO,FICKCI,TDCO,TDCI,PA_SAT
Z103426,F,1970-10-02,2.000000e+11,2020-01-09,4/1/20,2,1,NaN,1.0,NaN,...,56; 59,33; 33,40; 43,25,113.0,2.39,1.42,2.3,1.37,28
Z103697,F,1960-07-10,6.759402e+07,2016-09-12,9/24/16,2,1,NaN,1.0,NaN,...,67,36,48,36,NaN,2.9,1.52,2.99,1.57,41
Z1050590,F,1968-03-26,2.000000e+11,2020-12-02,12/2/20,2,1,NaN,NaN,NaN,...,101,39,62,10; 14; 8,100.0,3.8,2.22,5.17,3.02,59.1
Z1058186,M,1937-06-15,5.383417e+07,2014-09-25,10/11/14,2,2,NaN,1.0,NaN,...,56; 40,27; 16,37; 27,14; 28; 19,NaN,3.3800; 6.0800,1.8300; 3.2900,3.1800; 4.7200,1.7200; 2.5500,52.0000; 69.0000
Z1059364,F,1957-09-24,5.440898e+07,2014-10-27,11/20/14,2,1,NaN,1.0,NaN,...,50,17,33,18,NaN,4.35,2.16,4.16,2.07,63
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Z975360,M,1956-02-23,2.000000e+11,2018-01-18,3/26/18,5,1,NaN,NaN,1.0,...,67,34,49,37; 36,NaN,2.89,1.31,3.07,1.4,43
Z98220,M,1972-10-03,2.000000e+11,2019-08-28,9/4/19,3,1,NaN,1.0,NaN,...,91; 93,50; 55,64; 70,26,87.0,3.11,1.27,1.97,0.8,37.3
Z984151,M,1981-09-07,2.000000e+11,2019-02-25,3/27/19,3,1,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Z988224,M,1998-05-08,2.000000e+11,2017-10-19,11/27/17,3,1,NaN,1.0,1.0,...,22; 21,13; 12,17; 16,11,138.0,3.85,2.06,2.55,1.37,60.2


In [5]:
#Make date columns date objects
cath['ProcDate'] = pd.to_datetime(cath['ProcDate'])
shockOrig['PAT_DOB'] = pd.to_datetime(shockOrig['PAT_DOB'])
shockOrig['IndexAdmitDt'] = pd.to_datetime(shockOrig['IndexAdmitDt'])
shockOrig['CATH_DATE'] = pd.to_datetime(shockOrig['CATH_DATE'])
shockOrig['ADM1_AdmitDt'] = pd.to_datetime(shockOrig['ADM1_AdmitDt'])
shockOrig['ADM2_AdmitDt'] = pd.to_datetime(shockOrig['ADM2_AdmitDt'])
shockOrig['ADM3_AdmitDt'] = pd.to_datetime(shockOrig['ADM3_AdmitDt'])
shockOrig['EchoLowDate'] = pd.to_datetime(shockOrig['EchoLowDate'])
shockOrig['EchoHighDate'] = pd.to_datetime(shockOrig['EchoHighDate'])


#sort cath by index and date
cath = cath.sort_values(['PAT_ID', 'ProcDate'], ascending=[True, True])

#NOTE - TODO Keep this?????
#drop patients with heart TX
for idx in shockOrig.index:
    pt = shockOrig.loc[idx]
    
    if pt['HeartTx'] == 1:
        shockOrig = shockOrig.drop([idx])
        
        if idx in cath.index:
            cath = cath.drop([idx])
            
shockOrig

,PAT_SEX,PAT_DOB,IndexAdmissionCSN,IndexAdmitDt,IndexDischDt,IndexCathCount,Deceased,H2H,SystolicHF,DiastolicHF,...,PA_SYS,PA_DIA,PA_MEAN,PCW_MEAN,FICK_HR,FICKCO,FICKCI,TDCO,TDCI,PA_SAT
Z103426,F,1970-10-02,2.000000e+11,2020-01-09,4/1/20,2,1,NaN,1.0,NaN,...,56; 59,33; 33,40; 43,25,113.0,2.39,1.42,2.3,1.37,28
Z103697,F,1960-07-10,6.759402e+07,2016-09-12,9/24/16,2,1,NaN,1.0,NaN,...,67,36,48,36,NaN,2.9,1.52,2.99,1.57,41
Z1050590,F,1968-03-26,2.000000e+11,2020-12-02,12/2/20,2,1,NaN,NaN,NaN,...,101,39,62,10; 14; 8,100.0,3.8,2.22,5.17,3.02,59.1
Z1058186,M,1937-06-15,5.383417e+07,2014-09-25,10/11/14,2,2,NaN,1.0,NaN,...,56; 40,27; 16,37; 27,14; 28; 19,NaN,3.3800; 6.0800,1.8300; 3.2900,3.1800; 4.7200,1.7200; 2.5500,52.0000; 69.0000
Z1059364,F,1957-09-24,5.440898e+07,2014-10-27,11/20/14,2,1,NaN,1.0,NaN,...,50,17,33,18,NaN,4.35,2.16,4.16,2.07,63
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Z93479,M,1948-11-24,2.000000e+11,2017-06-30,7/5/17,2,2,NaN,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Z93572,M,1948-07-02,6.880236e+07,2016-11-08,11/8/16,2,2,NaN,1.0,NaN,...,48; 52,26; 27,35; 33,15; 15,77.0,4.7,2.19,3.97,1.85,67.4
Z937913,M,1980-08-24,2.000000e+11,2018-09-26,10/10/18,3,1,1.0,1.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Z958018,F,1960-07-30,6.569102e+07,2016-06-09,6/9/16,2,1,NaN,1.0,NaN,...,45,20,29,20,100.0,1.61,1.05,1.97,1.27,34.8


# HEMO

In [6]:
#go thru and append multiple rows as needed
giantList = []

for idx in shockOrig.index:
    lst = []
    if idx not in cath.index: #verifying no hemo recorded in shock dataset that not in cath
        pt = shockOrig.loc[idx]
        if type(pt) != pd.Series:
            pt = pt.iloc[0]
        if not pd.isna(pt['RA_MEAN']) or not pd.isna(pt['PA_SYS']) or not pd.isna(pt['TDCO']):
                print(idx, "-Found hemo not in cath dataset")
    
    else:
        pt = cath.loc[idx]
#         print(idx)
        
        #get age, gender, race
        shPt = shockOrig.loc[idx]
        dob = shPt['PAT_DOB']
        admDate = shPt['IndexAdmitDt']
        age = admDate.year - dob.year - ((admDate.month, admDate.day) < (dob.month, dob.day))
        
        if shPt['PAT_SEX'] == 'M': 
            gender = 1
        else:
            gender = 2
            
        race = np.nan
        
        #Ejf for dif admissions
        ef = shPt['ADM1_DischargeEF']
        if not pd.isna(ef):
            adm1EF = re.findall(r'\d+', ef)
            adm1EF = [float(i) for i in adm1EF]
            if len(adm1EF) > 1:
                adm1EF = sum(adm1EF) / len(adm1EF)
            else:
                adm1EF = adm1EF[0]
        else:
            adm1EF = np.nan
            
        ef = shPt['ADM2_DischargeEF']
        if not pd.isna(ef):
            adm2EF = re.findall(r'\d+', ef)
            adm2EF = [float(i) for i in adm2EF]
            if len(adm2EF) > 1:
                adm2EF = sum(adm2EF) / len(adm2EF)
            else:
                adm2EF = adm2EF[0]
        else:
            adm2EF = np.nan

        ef = shPt['ADM3_DischargeEF']
        if not pd.isna(ef):
            adm3EF = re.findall(r'\d+', ef)
            adm3EF = [float(i) for i in adm3EF]
            if len(adm3EF) > 1:
                adm3EF = sum(adm3EF) / len(adm3EF)
            else:
                adm3EF = adm3EF[0]
        else:
            adm3EF = np.nan
            
        #echo low and high
        ef = shPt['EchoLowValue']
        if not pd.isna(ef):
            echoLow = re.findall(r'\d+', ef)
            echoLow = [float(i) for i in echoLow]
            if len(echoLow) > 1:
                echoLow = sum(echoLow) / len(echoLow)
            else:
                echoLow = echoLow[0]
        else:
            echoLow = np.nan
            
        ef = shPt['EchoHighValue']
        if not pd.isna(ef):
            echoHigh = re.findall(r'\d+', ef)
            echoHigh = [float(i) for i in echoHigh]
            if len(echoHigh) > 1:
                echoHigh = sum(echoHigh) / len(echoHigh)
            else:
                echoHigh = echoHigh[0]
        else:
            echoHigh= np.nan

        dates = sorted(list(set(pt['ProcDate'])))
        for d in dates: #iterate through each timestamp w/recorded hemo
            ptDt = pt[pt['ProcDate'] == d]
            
            #get apropriate ejf and bps
            adms = []
            if not pd.isnull(shPt['ADM1_AdmitDt']):
                adms.append(shPt['ADM1_AdmitDt'])
            if not pd.isnull(shPt['ADM2_AdmitDt']):
                adms.append(shPt['ADM2_AdmitDt'])
            if not pd.isnull(shPt['ADM3_AdmitDt']):
                adms.append(shPt['ADM3_AdmitDt'])
                
            echoDates = []
            if not pd.isnull(shPt['EchoLowDate']):
                echoDates.append(shPt['EchoLowDate'])
            if not pd.isnull(shPt['EchoHighDate']):
                echoDates.append(shPt['EchoHighDate'])
                
            if adms == [] and echoDates == []: #if no adms and no echo
#                 print(idx, "has no admissions, and no echo")
                ejf = np.nan
                sys = np.nan
                dias = np.nan
            elif adms == []: #just no adms
#                 print(idx, "has no admissions, using echo")
                
                closest = min(echoDates, key=lambda x: abs(x - d))
                
                if closest == shPt['EchoLowDate']:
                    ejf = echoLow
                    sys = np.nan
                    dias = np.nan
                else:
                    ejf = echoHigh
                    sys = np.nan
                    dias = np.nan
            else: #do closest adms
                closest = min(adms, key=lambda x: abs(x - d))
                if closest == shPt['ADM1_AdmitDt']:
                    ejf = adm1EF
                    bp = shPt['ADM1_SysBP']
                    if not pd.isna(bp):
                        vals = str(bp).split('/')
                        sys = float(vals[0])
                        dias = float(vals[1])
                    else:
                        sys = np.nan
                        dias = np.nan
                elif closest == shPt['ADM2_AdmitDt']:
                    ejf = adm2EF
                    bp = shPt['ADM2_SysBP']
                    if not pd.isna(bp):
                        vals = str(bp).split('/')
                        sys = float(vals[0])
                        dias = float(vals[1])
                    else:
                        sys = np.nan
                        dias = np.nan
                else:
                    ejf = adm3EF
                    bp = shPt['ADM3_SysBP']
                    if not pd.isna(bp):
                        vals = str(bp).split('/')
                        sys = float(vals[0])
                        dias = float(vals[1])
                    else:
                        sys = np.nan
                        dias = np.nan
            
            val = ptDt[ptDt['Flo_Disply_Nme'] == 'RA Mean']['Msr_Val'].tolist()
            if len(val) == 1:
                rap = val[0]
            elif len(val) > 1:
                rap = sum(val) / len(val)
            else: #val == none
                rap = np.nan

            val = ptDt[ptDt['Flo_Disply_Nme'] == 'PA Sys']['Msr_Val'].tolist()
            if len(val) == 1:
                pas = val[0]
            elif len(val) > 1:
                pas = sum(val) / len(val)
            else: #val == none
                pas = np.nan
                
            val = ptDt[ptDt['Flo_Disply_Nme'] == 'PA Dia']['Msr_Val'].tolist()
            if val == []: 
                    val = ptDt[ptDt['Flo_Disply_Nme'] == 'PA DIa']['Msr_Val'].tolist()

            if len(val) == 1:
                pad = val[0]
            elif len(val) > 1:
                pad = sum(val) / len(val)
            else: #val == none
                pad = np.nan

            val = ptDt[ptDt['Flo_Disply_Nme'] == 'PA Mean']['Msr_Val'].tolist()
            if len(val) == 1:
                pamn = val[0]
            elif len(val) > 1:
                pamn = sum(val) / len(val)
            else: #val == none
                pamn = np.nan

            val = ptDt[ptDt['Flo_Disply_Nme'] == 'PCW Mean']['Msr_Val'].tolist()
            if len(val) == 1:
                pcwp = val[0]
            elif len(val) > 1:
                pcwp = sum(val) / len(val)
            else: #val == none
                pcwp = np.nan

            val = ptDt[ptDt['Flo_Disply_Nme'] == 'Thermal CO']['Msr_Val'].tolist()
            if val == []:
                val = ptDt[ptDt['Flo_Disply_Nme'] == 'Fick CO']['Msr_Val'].tolist()
            if len(val) == 1:
                co = val[0]
            elif len(val) > 1:
                co = sum(val) / len(val)
            else: #val == none
                co = np.nan

            val = ptDt[ptDt['Flo_Disply_Nme'] == 'Thermal CI']['Msr_Val'].tolist()
            if val == []:
                val = ptDt[ptDt['Flo_Disply_Nme'] == 'Fick CI']['Msr_Val'].tolist()
            if len(val) == 1:
                ci = val[0]
            elif len(val) > 1:
                ci = sum(val) / len(val)
            else: #val == none
                ci = np.nan

            val = ptDt[ptDt['Flo_Disply_Nme'] == 'PA Sat']['Msr_Val'].tolist()
            if len(val) == 1:
                mixed = val[0]
            elif len(val) > 1:
                mixed = sum(val) / len(val)
            else: #val == none
                mixed = np.nan
                
            val = ptDt[ptDt['Flo_Disply_Nme'] == 'Fick HR']['Msr_Val'].tolist()
            if len(val) == 1:
                hr = val[0]
            elif len(val) > 1:
                hr = sum(val) / len(val)
            else: #val == none
                hr = np.nan
                
            #Composite Hemo
            MAP = sys + ((2 * dias) / 3)
            MPAP = pas + ((2 * pad) / 3)
            CPI = (ci * MAP) / 451
            PP = sys - dias
            PPP = PP / sys
            PAPP = (pas  - pad) / pas
            SVR = 80 * (MAP  - rap) / co
            RAT = rap / pcwp
            PPRatio = PP / hr
            PAPi = (pas - pad) / rap if rap else 0
            SAPi = (sys - dias) / pcwp
            CPP = dias - pcwp
            PRAPRat = PP / rap if rap else 0

            lst = [idx, age, gender, race, ejf, rap, pas, pad, pamn, pcwp, co, ci, mixed, sys, dias, hr, MAP, MPAP, CPI,
                  PP, PPP, PAPP, SVR, RAT, PPRatio, PAPi, SAPi, CPP, PRAPRat]
            
            giantList.append(lst)
            
            

            
varz = ['Age', 'Gender', 'Race', 'EjF', 'RAP', 'PAS', 'PAD', 'PAMN', 'PCWP', 'CO', 
                            'CI', 'MIXED', 'BPSYS', 'BPDIAS', 'HRTRT', 'MAP','MPAP', 'CPI',
                            'PP', 'PPP', 'PAPP', 'SVR', 'RAT', 'PPRatio', 'PAPi', 'SAPi', 'CPP', 'PRAPRat']

varz.insert(0, 'ID')
hemoDF = pd.DataFrame(giantList, columns=varz).set_index('ID', drop=True)
hemoDF

,Age,Gender,Race,EjF,RAP,PAS,PAD,PAMN,PCWP,CO,...,PP,PPP,PAPP,SVR,RAT,PPRatio,PAPi,SAPi,CPP,PRAPRat
ID,,,,,,,,,,,,,,,,,,,,,
Z103426,49,2,NaN,17.5,25.5,53.750000,31.500000,40.25,25.333333,2.950,...,42.0,0.407767,0.413953,3204.519774,1.006579,0.365217,0.872549,1.657895,35.666667,1.647059
Z103426,49,2,NaN,17.5,15.0,42.333333,26.333333,33.00,21.400000,3.800,...,42.0,0.407767,0.377953,2708.771930,0.700935,0.368421,1.066667,1.962617,39.600000,2.800000
Z103697,56,2,NaN,22.5,24.0,67.000000,36.000000,48.00,36.000000,2.990,...,29.0,0.318681,0.462687,2898.550725,0.666667,NaN,1.291667,0.805556,26.000000,1.208333
Z103697,56,2,NaN,22.5,17.0,55.000000,23.000000,39.00,28.000000,3.710,...,29.0,0.318681,0.581818,2486.972147,0.607143,NaN,1.882353,1.035714,34.000000,1.705882
Z1050590,52,2,NaN,67.5,14.0,94.500000,36.000000,58.50,11.500000,4.685,...,NaN,NaN,0.619048,NaN,1.217391,NaN,4.178571,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Z958018,55,2,NaN,NaN,20.0,45.000000,20.000000,29.00,20.000000,1.970,...,45.0,0.412844,0.555556,5346.869712,1.000000,0.450000,1.250000,2.250000,44.000000,2.250000
Z958018,55,2,NaN,NaN,6.0,43.000000,19.000000,30.00,20.000000,3.200,...,45.0,0.412844,0.558140,3641.666667,0.300000,0.542169,4.000000,2.250000,44.000000,7.500000
Z958018,55,2,NaN,NaN,12.0,47.000000,26.000000,35.00,32.000000,0.930,...,45.0,0.412844,0.446809,12014.336918,0.375000,0.450000,1.750000,1.406250,32.000000,3.750000


In [7]:
#save to file
hemoDF.to_csv("../Data Validation/Serial Cardiac Caths/Original DataFrames/HemoSerialCardiac.csv")


In [8]:
#Make Hemo Labels
labels = pd.DataFrame()
indexes = sorted(list(set(hemoDF.index)))
labels['ID'] = indexes
labels = labels.set_index('ID')

#Death LVAD
lst = []
for idx in indexes:
    pt = shockOrig.loc[idx]
    if pt['Deceased'] == 2: 
        lst.append(1)
    else:
        lst.append(0)
labels['Death'] = lst

#Rehosp
rhlst = []
for idx in indexes:
    pt = shockOrig.loc[idx]
    if pd.isna(pt['ADM2_AdmitDt']): 
        rhlst.append(0)
    else:
        rhlst.append(1)
labels['Rehosp'] = rhlst


labels

,Death,Rehosp
ID,,
Z103426,0,0
Z103697,0,1
Z1050590,0,0
Z1058186,1,1
Z1059364,0,1
...,...,...
Z93479,1,1
Z93572,1,1
Z937913,0,1


In [9]:
labels.to_csv("../Data Validation/Serial Cardiac Caths/Original DataFrames/HemoLabelsSerialCardiac.csv")

# All Data

In [10]:
shock = shockOrig

cohort = pd.DataFrame()
cohort['ID'] = shock.index
cohort = cohort.set_index('ID')

lst = []
for idx in shock.index: #age at first admission
    pt = shock.loc[idx]
    
    dob = pt['PAT_DOB']
    admDate = pt['IndexAdmitDt']
    
    age = admDate.year - dob.year - ((admDate.month, admDate.day) < (dob.month, dob.day))
    lst.append(age)

cohort['Age'] = lst

lst = []
for idx in shock.index:
    pt = shock.loc[idx]
    if pt['PAT_SEX'] == 'M': 
        lst.append(1)
    else:
        lst.append(2)
cohort['Gender'] = lst

#will have 3 rows --> adm1, adm2 and adm3
cohort['BMI_adm1'] = shock['ADM1_BMI'].tolist()
cohort['BMI_adm2'] = shock['ADM2_BMI'].tolist()
cohort['BMI_adm3'] = shock['ADM3_BMI'].tolist()

#convert ejf from ranges to values for 3 adms
lst = []
for idx in shock.index:
    pt = shock.loc[idx]
    ef = pt['ADM1_DischargeEF']
    if not pd.isna(ef):
        val = re.findall(r'\d+', ef)
        val = [float(i) for i in val]
        if len(val) > 1:
            val = sum(val) / len(val)
        else:
            val = val[0]
    else:
        val = np.nan
        
    lst.append(val)
        
cohort['EjF_adm1'] = lst

lst = []
for idx in shock.index:
    pt = shock.loc[idx]
    ef = pt['ADM2_DischargeEF']
    if not pd.isna(ef):
        val = re.findall(r'\d+', ef)
        val = [float(i) for i in val]
        if len(val) > 1:
            val = sum(val) / len(val)
        else:
            val = val[0]
    else:
        val = np.nan
    lst.append(val)
        
cohort['EjF_adm2'] = lst

lst = []
for idx in shock.index:
    pt = shock.loc[idx]
    ef = pt['ADM3_DischargeEF']
    if not pd.isna(ef):
        val = re.findall(r'\d+', ef)
        val = [float(i) for i in val]
        if len(val) > 1:
            val = sum(val) / len(val)
        else:
            val = val[0]
    else:
        val = np.nan
    lst.append(val)
        
cohort['EjF_adm3'] = lst


#Initial Hosp Days
cohort['InitialHospDays_adm1'] = shock['ADM1_LOS'].tolist()
cohort['InitialHospDays_adm2'] = shock['ADM2_LOS'].tolist()
cohort['InitialHospDays_adm3'] = shock['ADM3_LOS'].tolist()

#Total Hosp days
lst = []
for idx in shock.index:
    pt = shock.loc[idx]
    a1 = pt['ADM1_LOS']
    a2 = pt['ADM2_LOS']
    a3 = pt['ADM3_LOS']
    if pd.isna(pt['ADM1_LOS']):
        a1 = 0
    if pd.isna(pt['ADM2_LOS']):
        a2 = 0
    if pd.isna(pt['ADM3_LOS']):
        a3 = 0
    
    los = a1 + a2 + a3
    lst.append(los)

cohort['TotalHospDays'] = lst
    
cohort

,Age,Gender,BMI_adm1,BMI_adm2,BMI_adm3,EjF_adm1,EjF_adm2,EjF_adm3,InitialHospDays_adm1,InitialHospDays_adm2,InitialHospDays_adm3,TotalHospDays
ID,,,,,,,,,,,,
Z103426,49,2,24.2,NaN,NaN,17.5,NaN,NaN,83.0,NaN,NaN,83.0
Z103697,56,2,NaN,NaN,NaN,22.5,NaN,NaN,12.0,4.0,NaN,16.0
Z1050590,52,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0
Z1058186,77,1,24.5,NaN,22.6,22.5,NaN,17.5,15.0,10.0,4.0,29.0
Z1059364,57,2,35.3,36.6,NaN,17.5,17.5,NaN,2.0,24.0,NaN,26.0
...,...,...,...,...,...,...,...,...,...,...,...,...
Z93479,68,1,31.0,32.3,31.1,NaN,22.5,32.5,4.0,7.0,12.0,23.0
Z93572,68,1,28.0,27.7,NaN,NaN,NaN,22.5,18.0,4.0,36.0,58.0
Z937913,38,1,39.5,NaN,37.8,NaN,15.0,15.0,14.0,5.0,4.0,23.0


In [11]:
lst = []
for idx in shock.index:
    pt = shock.loc[idx]
    if pt['COPD'] == 1: 
        lst.append(1)
    else:
        lst.append(0)
cohort['COPD'] = lst

lst = []
for idx in shock.index:
    pt = shock.loc[idx]
    if pt['DM'] == 1: 
        lst.append(1)
    else:
        lst.append(0)
cohort['DIAB'] = lst

lst = []
for idx in shock.index:
    pt = shock.loc[idx]
    if pt['CKD'] == 1: 
        lst.append(1)
    else:
        lst.append(0)
cohort['RENAL'] = lst

lst = []
for idx in shock.index:
    pt = shock.loc[idx]
    if pt['HeartTx'] == 1: 
        lst.append(1)
    else:
        lst.append(0)
cohort['HTRANS'] = lst

cohort.head(5)

,Age,Gender,BMI_adm1,BMI_adm2,BMI_adm3,EjF_adm1,EjF_adm2,EjF_adm3,InitialHospDays_adm1,InitialHospDays_adm2,InitialHospDays_adm3,TotalHospDays,COPD,DIAB,RENAL,HTRANS
ID,,,,,,,,,,,,,,,,
Z103426,49,2,24.2,NaN,NaN,17.5,NaN,NaN,83.0,NaN,NaN,83.0,0,1,0,0
Z103697,56,2,NaN,NaN,NaN,22.5,NaN,NaN,12.0,4.0,NaN,16.0,1,0,0,0
Z1050590,52,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0,0,0,0
Z1058186,77,1,24.5,NaN,22.6,22.5,NaN,17.5,15.0,10.0,4.0,29.0,0,1,1,0
Z1059364,57,2,35.3,36.6,NaN,17.5,17.5,NaN,2.0,24.0,NaN,26.0,0,0,0,0


In [12]:
# Labs
cohort['BUN_adm1'] = shock['ADM1_Labs_BUN'].tolist()
cohort['BUN_adm2'] = shock['ADM2_Labs_BUN'].tolist()
cohort['BUN_adm3'] = shock['ADM3_Labs_BUN'].tolist()

cohort['CRT_adm1'] = shock['ADM1_Labs_Cr'].tolist()
cohort['CRT_adm2'] = shock['ADM2_Labs_Cr'].tolist()
cohort['CRT_adm3'] = shock['ADM3_Labs_Cr'].tolist()

cohort['SOD_adm1'] = shock['ADM1_Labs_NA'].tolist()
cohort['SOD_adm2'] = shock['ADM2_Labs_NA'].tolist()
cohort['SOD_adm3'] = shock['ADM3_Labs_NA'].tolist()

cohort['WBC_adm1'] = shock['ADM1_Labs_Lymphocytes'].tolist()
cohort['WBC_adm2'] = shock['ADM2_Labs_Lymphocytes'].tolist()
cohort['WBC_adm3'] = shock['ADM3_Labs_Lymphocytes'].tolist()

#Meds
#ACE
lst = []
for idx in shock.index:
    pt = shock.loc[idx]
    if pt['ADM1_Antihypertensive__ACE Inhibitors'] == 1 or pt['ADM1_Antihypertensive__Angiotensin II Receptor Antagonists'] == 1 or pt['ADM1_Antihypertensive__Selective Aldosterone Receptor Antagonists (SARAs)'] == 1: 
        lst.append(1)
    else:
        lst.append(0)
        
cohort['ACE_adm1'] = lst

lst = []
for idx in shock.index:
    pt = shock.loc[idx]
    if pt['ADM2_Antihypertensive__ACE Inhibitors'] == 1 or pt['ADM2_Antihypertensive__Angiotensin II Receptor Antagonists'] == 1 or pt['ADM2_Antihypertensive__Selective Aldosterone Receptor Antagonists (SARAs)'] == 1: 
        lst.append(1)
    else:
        lst.append(0)
        
cohort['ACE_adm2'] = lst

lst = []
for idx in shock.index:
    pt = shock.loc[idx]
    if pt['ADM3_Antihypertensive__ACE Inhibitors'] == 1 or pt['ADM3_Antihypertensive__Angiotensin II Receptor Antagonists'] == 1 or pt['ADM3_Antihypertensive__Selective Aldosterone Receptor Antagonists (SARAs)'] == 1: 
        lst.append(1)
    else:
        lst.append(0)
        
cohort['ACE_adm3'] = lst

#BET
lst = []
for idx in shock.index:
    pt = shock.loc[idx]
    if pt['ADM1_Beta Blockers__Beta Blocker Combinations'] == 1 or pt['ADM1_Beta Blockers__Beta Blockers Cardio-Selective'] == 1 or pt['ADM1_Beta Blockers__Beta Blockers Non-Selective'] == 1: 
        lst.append(1)
    else:
        lst.append(0)
        
cohort['BET_adm1'] = lst

lst = []
for idx in shock.index:
    pt = shock.loc[idx]
    if pt['ADM2_Beta Blockers__Beta Blocker Combinations'] == 1 or pt['ADM2_Beta Blockers__Beta Blockers Cardio-Selective'] == 1 or pt['ADM2_Beta Blockers__Beta Blockers Non-Selective'] == 1: 
        lst.append(1)
    else:
        lst.append(0)
        
cohort['BET_adm2'] = lst

lst = []
for idx in shock.index:
    pt = shock.loc[idx]
    if pt['ADM3_Beta Blockers__Beta Blocker Combinations'] == 1 or pt['ADM3_Beta Blockers__Beta Blockers Cardio-Selective'] == 1 or pt['ADM3_Beta Blockers__Beta Blockers Non-Selective'] == 1: 
        lst.append(1)
    else:
        lst.append(0)
        
cohort['BET_adm3'] = lst

#DIUR
cohort['DIUR_adm1'] = shock['ADM1_Diuretics__Potassium Sparing Diuretics'].tolist()
cohort['DIUR_adm2'] = shock['ADM2_Diuretics__Potassium Sparing Diuretics'].tolist()
cohort['DIUR_adm3'] = shock['ADM3_Diuretics__Potassium Sparing Diuretics'].tolist()

cohort

,Age,Gender,BMI_adm1,BMI_adm2,BMI_adm3,EjF_adm1,EjF_adm2,EjF_adm3,InitialHospDays_adm1,InitialHospDays_adm2,...,WBC_adm3,ACE_adm1,ACE_adm2,ACE_adm3,BET_adm1,BET_adm2,BET_adm3,DIUR_adm1,DIUR_adm2,DIUR_adm3
ID,,,,,,,,,,,,,,,,,,,,,
Z103426,49,2,24.2,NaN,NaN,17.5,NaN,NaN,83.0,NaN,...,NaN,1,0,0,1,0,0,1.0,1.0,NaN
Z103697,56,2,NaN,NaN,NaN,22.5,NaN,NaN,12.0,4.0,...,NaN,1,1,0,1,1,0,1.0,1.0,NaN
Z1050590,52,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0,0,0,0,0,0,NaN,NaN,NaN
Z1058186,77,1,24.5,NaN,22.6,22.5,NaN,17.5,15.0,10.0,...,0.57,0,0,0,0,1,1,NaN,NaN,NaN
Z1059364,57,2,35.3,36.6,NaN,17.5,17.5,NaN,2.0,24.0,...,NaN,1,1,0,1,1,0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Z93479,68,1,31.0,32.3,31.1,NaN,22.5,32.5,4.0,7.0,...,1.36,0,0,0,0,0,0,1.0,1.0,1.0
Z93572,68,1,28.0,27.7,NaN,NaN,NaN,22.5,18.0,4.0,...,1.78,0,0,0,1,1,1,NaN,NaN,NaN
Z937913,38,1,39.5,NaN,37.8,NaN,15.0,15.0,14.0,5.0,...,NaN,1,0,1,0,0,0,1.0,1.0,1.0


In [13]:
#Invasive Hemo
# cohort['HRTRT_adm1'] = shock['ADM1_EKG_QRSD'].tolist()
# cohort['HRTRT_adm2'] = shock['ADM2_EKG_QRSD'].tolist()
# cohort['HRTRT_adm3'] = shock['ADM3_EKG_QRSD'].tolist()

lst = []
for idx in shock.index:
    pt = shock.loc[idx]['ADM1_EKG_QRSD']
    if pd.isna(pt):
        lst.append(np.nan)
    else:
        lst.append(float(pt))

cohort['HRTRT_adm1'] = lst

lst = []
for idx in shock.index:
    pt = shock.loc[idx]['ADM2_EKG_QRSD']
    if pd.isna(pt):
        lst.append(np.nan)
    else:
        lst.append(float(pt))

cohort['HRTRT_adm2'] = lst

lst = []
for idx in shock.index:
    pt = shock.loc[idx]['ADM3_EKG_QRSD']
    if pd.isna(pt):
        lst.append(np.nan)
    else:
        lst.append(float(pt))

cohort['HRTRT_adm3'] = lst

sys = []
dias = []
for idx in shock.index:
    pt = shock.loc[idx]
    bp = pt['ADM1_SysBP']

    if not pd.isna(bp):
        vals = str(bp).split('/')
        sys.append(float(vals[0]))
        dias.append(float(vals[1]))
    else:
        sys.append(np.nan)
        dias.append(np.nan)
        
cohort['BPSYS_adm1'] = sys
cohort['BPDIAS_adm1'] = dias

sys = []
dias = []
for idx in shock.index:
    pt = shock.loc[idx]
    bp = pt['ADM2_SysBP']

    if not pd.isna(bp):
        vals = str(bp).split('/')
        sys.append(float(vals[0]))
        dias.append(float(vals[1]))
    else:
        sys.append(np.nan)
        dias.append(np.nan)
        
cohort['BPSYS_adm2'] = sys
cohort['BPDIAS_adm2'] = dias

sys = []
dias = []
for idx in shock.index:
    pt = shock.loc[idx]
    bp = pt['ADM3_SysBP']

    if not pd.isna(bp):
        vals = str(bp).split('/')
        sys.append(float(vals[0]))
        dias.append(float(vals[1]))
    else:
        sys.append(np.nan)
        dias.append(np.nan)
        
cohort['BPSYS_adm3'] = sys
cohort['BPDIAS_adm3'] = dias

cohort

,Age,Gender,BMI_adm1,BMI_adm2,BMI_adm3,EjF_adm1,EjF_adm2,EjF_adm3,InitialHospDays_adm1,InitialHospDays_adm2,...,DIUR_adm3,HRTRT_adm1,HRTRT_adm2,HRTRT_adm3,BPSYS_adm1,BPDIAS_adm1,BPSYS_adm2,BPDIAS_adm2,BPSYS_adm3,BPDIAS_adm3
ID,,,,,,,,,,,,,,,,,,,,,
Z103426,49,2,24.2,NaN,NaN,17.5,NaN,NaN,83.0,NaN,...,NaN,63.0,NaN,NaN,103.0,61.0,NaN,NaN,NaN,NaN
Z103697,56,2,NaN,NaN,NaN,22.5,NaN,NaN,12.0,4.0,...,NaN,134.0,152.0,NaN,91.0,62.0,98.0,70.0,NaN,NaN
Z1050590,52,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Z1058186,77,1,24.5,NaN,22.6,22.5,NaN,17.5,15.0,10.0,...,NaN,120.0,130.0,146.0,82.0,50.0,98.0,56.0,91.0,60.0
Z1059364,57,2,35.3,36.6,NaN,17.5,17.5,NaN,2.0,24.0,...,NaN,92.0,90.0,NaN,105.0,76.0,134.0,65.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Z93479,68,1,31.0,32.3,31.1,NaN,22.5,32.5,4.0,7.0,...,1.0,118.0,160.0,144.0,82.0,0.0,80.0,0.0,78.0,0.0
Z93572,68,1,28.0,27.7,NaN,NaN,NaN,22.5,18.0,4.0,...,NaN,108.0,105.0,102.0,114.0,65.0,115.0,61.0,126.0,79.0
Z937913,38,1,39.5,NaN,37.8,NaN,15.0,15.0,14.0,5.0,...,1.0,122.0,115.0,82.0,76.0,0.0,97.0,50.0,124.0,55.0


In [14]:
#Composite Hemo

idx = sorted(set(cohort.index))

#MAP
a1 = []
a2 = []
a3 = []
for i in idx:
    t = cohort.loc[i]
    
    admit1 = t['BPSYS_adm1'] + ((2 * t['BPDIAS_adm1']) / 3)
    admit2 = t['BPSYS_adm2'] + ((2 * t['BPDIAS_adm2']) / 3)
    admit3 = t['BPSYS_adm3'] + ((2 * t['BPDIAS_adm3']) / 3)

    a1.append(admit1)
    a2.append(admit2)
    a3.append(admit3)
    
cohort['MAP_adm1'] = a1
cohort['MAP_adm2'] = a2
cohort['MAP_adm3'] = a3
    
#PP
a1 = []
a2 = []
a3 = []
for i in idx:
    t = cohort.loc[i]

    admit1 = t['BPSYS_adm1'] - t['BPDIAS_adm1']
    admit2 = t['BPSYS_adm2'] - t['BPDIAS_adm2']
    admit3 = t['BPSYS_adm3'] - t['BPDIAS_adm3']

    a1.append(admit1)
    a2.append(admit2)
    a3.append(admit3)
    
cohort['PP_adm1'] = a1
cohort['PP_adm2'] = a2
cohort['PP_adm3'] = a3

#PPP
a1 = []
a2 = []
a3 = []
for i in idx:
    t = cohort.loc[i]

    admit1 = t['PP_adm1'] / t['BPSYS_adm1']
    admit2 = t['PP_adm2'] / t['BPSYS_adm2']
    admit3 = t['PP_adm3'] / t['BPSYS_adm3']

    a1.append(admit1)
    a2.append(admit2)
    a3.append(admit3)
    
cohort['PPP_adm1'] = a1
cohort['PPP_adm2'] = a2
cohort['PPP_adm3'] = a3

#PPRatio
a1 = []
a2 = []
a3 = []
for i in idx:
    t = cohort.loc[i]

    admit1 = t['PP_adm1'] / t['HRTRT_adm1']
    admit2 = t['PP_adm2'] / t['HRTRT_adm2']
    admit3 = t['PP_adm3'] / t['HRTRT_adm3']

    a1.append(admit1)
    a2.append(admit2)
    a3.append(admit3)
    
cohort['PPRatio_adm1'] = a1
cohort['PPRatio_adm2'] = a2
cohort['PPRatio_adm3'] = a3

cohort

,Age,Gender,BMI_adm1,BMI_adm2,BMI_adm3,EjF_adm1,EjF_adm2,EjF_adm3,InitialHospDays_adm1,InitialHospDays_adm2,...,MAP_adm3,PP_adm1,PP_adm2,PP_adm3,PPP_adm1,PPP_adm2,PPP_adm3,PPRatio_adm1,PPRatio_adm2,PPRatio_adm3
ID,,,,,,,,,,,,,,,,,,,,,
Z103426,49,2,24.2,NaN,NaN,17.5,NaN,NaN,83.0,NaN,...,NaN,42.0,NaN,NaN,0.407767,NaN,NaN,0.666667,NaN,NaN
Z103697,56,2,NaN,NaN,NaN,22.5,NaN,NaN,12.0,4.0,...,NaN,29.0,28.0,NaN,0.318681,0.285714,NaN,0.216418,0.184211,NaN
Z1050590,52,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Z1058186,77,1,24.5,NaN,22.6,22.5,NaN,17.5,15.0,10.0,...,131.000000,32.0,42.0,31.0,0.390244,0.428571,0.340659,0.266667,0.323077,0.212329
Z1059364,57,2,35.3,36.6,NaN,17.5,17.5,NaN,2.0,24.0,...,NaN,29.0,69.0,NaN,0.276190,0.514925,NaN,0.315217,0.766667,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Z93479,68,1,31.0,32.3,31.1,NaN,22.5,32.5,4.0,7.0,...,78.000000,82.0,80.0,78.0,1.000000,1.000000,1.000000,0.694915,0.500000,0.541667
Z93572,68,1,28.0,27.7,NaN,NaN,NaN,22.5,18.0,4.0,...,178.666667,49.0,54.0,47.0,0.429825,0.469565,0.373016,0.453704,0.514286,0.460784
Z937913,38,1,39.5,NaN,37.8,NaN,15.0,15.0,14.0,5.0,...,160.666667,76.0,47.0,69.0,1.000000,0.484536,0.556452,0.622951,0.408696,0.841463


In [15]:
#Make separate datasets

#All Data - conditions
varz = ['Age', 'Gender', 'Race', 'Wt', 'BMI', 
       'InitialHospDays', 'TotalHospDays', 'NYHA', 'MLHFS', 'AF',
       'AlchE', 'ANGP', 'ARRH', 'CARREST', 'CVD', 'COPD', 'DEPR', 'DIAB',
       'GOUT', 'HEPT', 'HTN', 'MALIG', 'RENAL', 'SMOKING', 'STERD',
       'StrokeTIA', 'VAHD', 'VF', 'VHD', 'VT', 'ISCH', 'NonISCH', 'CABG',
       'HTRANS', 'ICD', 'PACE', 'PTCI', 'SixFtWlk', 'VO2','ALB', 'ALT', 'AST', 'BUN',
       'CRT', 'DIAL', 'HEC', 
       'HEM','PLA', 'POT',  'SOD',
       'TALB', 'TOTP', 'WBC', 'ACE','BET', 'NIT', 'DIUR',
       'EjF', 'BPDIAS', 'BPSYS', 'HR',
       'PV', 'MAP', 'PP', 'PPP', 'PPRatio']



giantList = []
for i in range(len(cohort)):
    df = cohort.iloc[i]
    row0 = []
    row1 = []
    row2 = []
    row0.append(df.name)
    row1.append(df.name)
    row2.append(df.name)
    for col in varz:
        if col in cohort.columns: #value for both
            row0.append(df[col])
            row1.append(df[col])
            row2.append(df[col])
        elif col + "_adm1" in cohort.columns and col + "_adm2" in cohort.columns and col + "_adm3" in cohort.columns:
            row0.append(df[col+"_adm1"])
            row1.append(df[col+"_adm2"])
            row2.append(df[col+"_adm3"])
        else:
            row0.append(0)
            row1.append(0)
            row2.append(0)
    
    giantList.append(row0)
    giantList.append(row1)
    giantList.append(row2)

varz.insert(0, 'ID')
allData = pd.DataFrame(giantList, columns=varz).set_index('ID', drop=True)
    
allData

,Age,Gender,Race,Wt,BMI,InitialHospDays,TotalHospDays,NYHA,MLHFS,AF,...,DIUR,EjF,BPDIAS,BPSYS,HR,PV,MAP,PP,PPP,PPRatio
ID,,,,,,,,,,,,,,,,,,,,,
Z103426,49.0,2.0,0,0,24.2,83.0,83.0,0,0,0,...,1.0,17.5,61.0,103.0,0,0,143.666667,42.0,0.407767,0.666667
Z103426,49.0,2.0,0,0,NaN,NaN,83.0,0,0,0,...,1.0,NaN,NaN,NaN,0,0,NaN,NaN,NaN,NaN
Z103426,49.0,2.0,0,0,NaN,NaN,83.0,0,0,0,...,NaN,NaN,NaN,NaN,0,0,NaN,NaN,NaN,NaN
Z103697,56.0,2.0,0,0,NaN,12.0,16.0,0,0,0,...,1.0,22.5,62.0,91.0,0,0,132.333333,29.0,0.318681,0.216418
Z103697,56.0,2.0,0,0,NaN,4.0,16.0,0,0,0,...,1.0,NaN,70.0,98.0,0,0,144.666667,28.0,0.285714,0.184211
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Z958018,55.0,2.0,0,0,16.1,14.0,30.0,0,0,0,...,1.0,NaN,64.0,109.0,0,0,151.666667,45.0,0.412844,0.266272
Z958018,55.0,2.0,0,0,19.1,2.0,30.0,0,0,0,...,NaN,37.5,52.0,83.0,0,0,117.666667,31.0,0.373494,0.191358
Z98220,46.0,1.0,0,0,27.1,4.0,16.0,0,0,0,...,1.0,NaN,89.0,128.0,0,0,187.333333,39.0,0.304688,0.286765


In [16]:
#save to file
allData.to_csv("../Data Validation/Serial Cardiac Caths/Original DataFrames/AllDataSerialCardiac.csv")

In [17]:
#Make Labels
labels = pd.DataFrame()
labels['ID'] = shock.index
labels = labels.set_index('ID')

#Death LVAD
lst = []
for idx in shock.index:
    pt = shock.loc[idx]
    if pt['Deceased'] == 2: 
        lst.append(1)
    else:
        lst.append(0)
labels['Death'] = lst

#Rehosp
rhlst = []
for idx in shock.index:
    pt = shock.loc[idx]
    if pd.isna(pt['ADM2_AdmitDt']): 
        rhlst.append(0)
    else:
        rhlst.append(1)
labels['Rehosp'] = rhlst


labels

,Death,Rehosp
ID,,
Z103426,0,0
Z103697,0,1
Z1050590,0,0
Z1058186,1,1
Z1059364,0,1
...,...,...
Z93479,1,1
Z93572,1,1
Z937913,0,1


In [18]:
labels.to_csv("../Data Validation/Serial Cardiac Caths/Original DataFrames/LabelsSerialCardiac.csv")